In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import xarray as xr

In [ ]:
# replace the file path below by the path of your input file. 
# If it is correct you can also unquote the line below the current line
fn = r'/home/hcwinsemius/Barotse/zambezi_4km/inmaps/forcing-2000_2018.nc'
# fn = r'c:\zambezi_4km\inmaps\forcing-2000_2018.nc'
ds = xr.open_dataset(fn)
ds

In [ ]:
# first drop temperature from dataset and get the sum of the two fluxes P (precip) and PET (potential evaporation)
ds_p_year = ds.drop('TEMP').groupby('time.year').sum('time')
# then drop PET and P and take the mean of temperature (in deg. Celsius)
ds_t_year = ds.drop(['PET', 'P']).groupby('time.year').mean('time')


In [ ]:
# recombine the outcoming datasets
ds = ds_p_year.merge(ds_t_year)

# let's mask any zero values by leaving values where they are above zero
ds['P'] = ds['P'].where(ds['P']>0)
ds['PET'] = ds['PET'].where(ds['PET']>0)
ds['TEMP'] = ds['TEMP'].where(ds['TEMP']>0)
ds['TEMP'][0].plot()  # check if this worked in a quick plot

In [ ]:
# store annual values in a new file
ds.to_netcdf('forcing_yearly.nc')

Let's inspect the annual total rainfall and potential evaporation

In [ ]:
# plot annual values of precip. col selects which dimension to use per subplot. This is super easy with xarray
# col_wrap selects the amount of columns to plot
ds['P'].plot(col='year', col_wrap=5, extend='max', cmap='terrain_r')

In [ ]:
ds['PET'].plot(col='year', col_wrap=5, extend='max', cmap='rainbow')

The results look as follows:
- annual precip is and potential evaporation look reasonable. I find PET a little bit lower than expected. But not serious
- there is significant interannual variability in precip and pot evapo
- years with low rainfall concur with years with high pot. evapo.
Below also a plot of long-term annual averages

In [ ]:
n = 0
f= plt.figure(figsize=(16, 6))
plt.subplot(121)
ds['P'][4].plot(cmap='terrain_r')
plt.subplot(122)
ds['PET'][4].plot(cmap='rainbow')

We may decide to bias correct potential evapo by a factor 1.15 or so. Let's also make an estimate of the cell-by-cell annual runoff coefficient using the Budyko framework (see https://www.hydrol-earth-syst-sci.net/23/4983/2019/ Table 1, for the equation.)

In [ ]:
import numpy as np
plt.figure(figsize=(16,13))
P_mean = ds['P'].mean(dim='year')
PET_mean = ds['PET'].mean(dim='year')

aridity = PET_mean/P_mean
# aridity.plot()

Ea_P = (aridity*np.tanh(1/aridity)*(1-np.exp(-aridity)))**0.5
# Ea_P.plot()
runoff_coeff = 1-Ea_P
runoff_coeff.plot(cmap='nipy_spectral_r')

So we see that runoff coefficients very from about 0.1 in the very South to about 0.4 in the very North. I find 0.4 quite high for a deciduous forest, so I still guess potential evapo is on the low side.